## Solution Approach Taken
From my perspective, I saw the task as semantic similarity rpoblem.Here, the task is to check if the questions of the test dataset is similar to the train dataset or not. Although, many matric of similarity can be checked, I chose the cosine similarity. The main obstacle here is that the number of data in the train set is too small to fit through any classifier as it will underfit or overfit depending on the hyperparameter. Therefore I refrained from building any sequential model model but relied on pretrained language models encoding (DistillBERT) which provided the best reslt.

In [2]:
# Inport All Libreries
!pip install nltk
import pandas as pd
import warnings
from sklearn.metrics import classification_report
import numpy
from gensim import corpora
import pprint
import random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import gensim 
from gensim.parsing.preprocessing import remove_stopwords
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore') 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#import train dataset
train_df = pd.read_csv("/content/FAQs.csv")
train_df

,Question,Answer
0,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.
1,Where was he born?,"He was born in Ulm, Germany."
2,When did he die?,"He died 18 April 1955 in Princeton, New Jersey..."
3,Who were his parents?,His father was Hermann Einstein and his mother...
4,Did he have any sisters and brothers?,He had one sister named Maja.
5,Did he marry and have children?,He was married to Mileva Marić between 1903 an...
6,Where did he receive his education?,He received his main education at the followin...
7,When was Albert Einstein awarded the Nobel Pri...,"The Nobel Prize Awarding Institution, the Roya..."
8,Did Albert Einstein attend the Nobel Prize Awa...,The Nobel Prize was announced on 9 November 19...
9,For what did he receive the Nobel Prize?,Einstein was rewarded for his many contributio...


In [4]:
#import test dataset
test_df = pd.read_csv("/content/FAQs_test.csv")
test_df

,Question
0,What is the date of his death?
1,Did Einstein have siblings?
2,Who was his wife?
3,What was Einstein's father's name?
4,At what institutions did he study?


# Baselines (Random Classification)

In [5]:
# first created the baseline result To compare other models 
answer_list_for_test_dataframe = [2,4,5,3,6]
y_ture = answer_list_for_test_dataframe
y_pred = [random.randint(0, 9) for number in range(5)]
print(classification_report(y_ture, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         1
           3       1.00      1.00      1.00         1
           4       0.00      0.00      0.00         1
           5       1.00      1.00      1.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         0

    accuracy                           0.40         5
   macro avg       0.29      0.29      0.29         5
weighted avg       0.40      0.40      0.40         5



## FAQ With Cosine Similarity (Removing Stopwords)

In [6]:
#cleaning the dataset
def clean_sentence(sentence, stp_rem =False):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    sentence = remove_stopwords(sentence) if stp_rem==True else sentence
    return sentence

def get_cleaned_sentences(df):
    return [clean_sentence(sentences) for sentences in df["Question"]]

cleaned_sentences=get_cleaned_sentences(train_df)
print(cleaned_sentences);

original_sentence  =  list(train_df["Question"])
original_sentence

['when was albert einstein born', 'where was he born', 'when did he die', 'who were his parents', 'did he have any sisters and brothers', 'did he marry and have children', 'where did he receive his education', 'when was albert einstein awarded the nobel prize in physics', 'did albert einstein attend the nobel prize award ceremony', 'for what did he receive the nobel prize']


['When was Albert Einstein born?',
 'Where was he born?',
 'When did he die?',
 'Who were his parents?',
 'Did he have any sisters and brothers?',
 'Did he marry and have children?',
 'Where did he receive his education?',
 'When was Albert Einstein awarded the Nobel Prize in Physics?',
 'Did Albert Einstein attend the Nobel Prize Award Ceremony?',
 'For what did he receive the Nobel Prize?']

In [7]:
#apply bag_of_words and embedding
sentence_words = [[word for word in sentence.split() ] for sentence in cleaned_sentences]
dictionary = corpora.Dictionary(sentence_words)
for key, value in dictionary.items(): print(key, ' : ', value)


bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(cleaned_sentences,bow_corpus): print(f'{sent} : {embedding}')


print('Test Dataset Embeddings')
for question in test_df['Question']:
  ques = clean_sentence(question);
  question_embedding = dictionary.doc2bow(question.split())
  print(f'{ques} : {question_embedding}')

0  :  albert
1  :  born
2  :  einstein
3  :  was
4  :  when
5  :  he
6  :  where
7  :  did
8  :  die
9  :  his
10  :  parents
11  :  were
12  :  who
13  :  and
14  :  any
15  :  brothers
16  :  have
17  :  sisters
18  :  children
19  :  marry
20  :  education
21  :  receive
22  :  awarded
23  :  in
24  :  nobel
25  :  physics
26  :  prize
27  :  the
28  :  attend
29  :  award
30  :  ceremony
31  :  for
32  :  what
when was albert einstein born : [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]
where was he born : [(1, 1), (3, 1), (5, 1), (6, 1)]
when did he die : [(4, 1), (5, 1), (7, 1), (8, 1)]
who were his parents : [(9, 1), (10, 1), (11, 1), (12, 1)]
did he have any sisters and brothers : [(5, 1), (7, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1)]
did he marry and have children : [(5, 1), (7, 1), (13, 1), (16, 1), (18, 1), (19, 1)]
where did he receive his education : [(5, 1), (6, 1), (7, 1), (9, 1), (20, 1), (21, 1)]
when was albert einstein awarded the nobel prize in physics : [(0, 1), 

In [8]:
# predict model with cosine similarity
y_pred = []
def retrieveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf,sentences):
    if len(question_embedding) == 0:
      question_embedding = [[(0,0)]]
    print(question_embedding)
    max_sim=-1;
    index_sim=-1;
    for index,faq_embedding in enumerate(sentence_embeddings):
        sim=cosine_similarity(faq_embedding,question_embedding)[0][0]
        print(index, sim, sentences[index])
        if sim>max_sim:
            max_sim=sim
            index_sim=index
    y_pred.append(index_sim)
       
    print("Question: ",question)
    print("Retrieved: ",FAQdf.iloc[index_sim,0]) 
    print(FAQdf.iloc[index_sim,1])        

for question in test_df['Question']:    
  retrieveAndPrintFAQAnswer(dictionary.doc2bow(clean_sentence(question).split()),bow_corpus,train_df,cleaned_sentences)

[(9, 1), (27, 1), (32, 1)]
0 0.11043152607484653 when was albert einstein born
1 0.7808688094430302 where was he born
2 0.9909924304103231 when did he die
3 0.9999999999999998 who were his parents
4 0.996240588195683 did he have any sisters and brothers
5 0.996240588195683 did he marry and have children
6 0.996240588195683 where did he receive his education
7 0.11043152607484653 when was albert einstein awarded the nobel prize in physics
8 0.11043152607484653 did albert einstein attend the nobel prize award ceremony
9 0.996240588195683 for what did he receive the nobel prize
Question:  What is the date of his death?
Retrieved:  Who were his parents?
His father was Hermann Einstein and his mother was Pauline Einstein (born Koch).
[(2, 1), (7, 1), (16, 1)]
0 0.4472135954999579 when was albert einstein born
1 0.9486832980505137 where was he born
2 0.9761870601839527 when did he die
3 0.93834311681711 who were his parents
4 0.9647638212377321 did he have any sisters and brothers
5 0.964763

In [9]:
print(classification_report(y_ture, y_pred))

              precision    recall  f1-score   support

           2       0.00      0.00      0.00       1.0
           3       0.00      0.00      0.00       1.0
           4       0.00      0.00      0.00       1.0
           5       0.00      0.00      0.00       1.0
           6       0.00      0.00      0.00       1.0

    accuracy                           0.00       5.0
   macro avg       0.00      0.00      0.00       5.0
weighted avg       0.00      0.00      0.00       5.0



## Cosine Similarity With Bag-of-Words With Answer Insted Of Questions

In [10]:
def get_cleaned_sentences_answer(df):
    return [clean_sentence(sentences) for sentences in df["Answer"]]

cleaned_sentences=get_cleaned_sentences_answer(train_df)
print(cleaned_sentences);

original_sentence  =  list(train_df["Answer"])
original_sentence

['albert einstein was born on 14 march 1879', 'he was born in ulm germany', 'he died 18 april 1955 in princeton new jersey usa', 'his father was hermann einstein and his mother was pauline einstein born koch', 'he had one sister named maja', 'he was married to mileva mari between 1903 and 1919 they had three children lieserl born 1902 hans albert born 1904 and eduard born 1910 he married elsa lwenthal in 1919 and they lived together until her death in 1936', 'he received his main education at the following schools catholic elementary school in munich germany 18851888luitpold gymnasium in munich germany 18881894 cantonal school in aarau switzerland 18951896 swiss federal institute of technology in zurich switzerland 18961900 phd from zurich university switzerland 1905', 'the nobel prize awarding institution the royal swedish academy of sciences decided to reserve the nobel prize in physics in 1921 and therefore no physics prize was awarded that year according to the statutes a reserved 

['Albert Einstein was born on 14 March 1879.',
 'He was born in Ulm, Germany. ',
 'He died 18 April 1955 in Princeton, New Jersey, USA.',
 'His father was Hermann Einstein and his mother was Pauline Einstein (born Koch).',
 'He had one sister named Maja.',
 'He was married to Mileva Marić between 1903 and 1919. They had three children, Lieserl (born 1902), Hans Albert (born 1904) and Eduard (born 1910). He married Elsa Löwenthal in 1919 and they lived together until her death in 1936.',
 'He received his main education at the following schools: Catholic elementary school in Munich, Germany (1885-1888)Luitpold Gymnasium in Munich, Germany (1888-1894) Cantonal school in Aarau, Switzerland (1895-1896) Swiss Federal Institute of Technology in Zurich, Switzerland (1896-1900) Ph.D. from Zurich University, Switzerland (1905)',
 'The Nobel Prize Awarding Institution, the Royal Swedish Academy of Sciences, decided to reserve the Nobel Prize in Physics in 1921, and therefore no Physics Prize was

In [11]:
import numpy
from gensim import corpora
import pprint

sentence_words = [[word for word in sentence.split() ] for sentence in cleaned_sentences]
dictionary = corpora.Dictionary(sentence_words)
for key, value in dictionary.items(): print(key, ' : ', value)


bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(cleaned_sentences,bow_corpus): print(f'{sent} : {embedding}')


print('Test Dataset Embeddings')
for question in test_df['Question']:
  ques = clean_sentence(question);
  question_embedding = dictionary.doc2bow(question.split())
  print(f'{ques} : {question_embedding}')

0  :  14
1  :  1879
2  :  albert
3  :  born
4  :  einstein
5  :  march
6  :  on
7  :  was
8  :  germany
9  :  he
10  :  in
11  :  ulm
12  :  18
13  :  1955
14  :  april
15  :  died
16  :  jersey
17  :  new
18  :  princeton
19  :  usa
20  :  and
21  :  father
22  :  hermann
23  :  his
24  :  koch
25  :  mother
26  :  pauline
27  :  had
28  :  maja
29  :  named
30  :  one
31  :  sister
32  :  1902
33  :  1903
34  :  1904
35  :  1910
36  :  1919
37  :  1936
38  :  between
39  :  children
40  :  death
41  :  eduard
42  :  elsa
43  :  hans
44  :  her
45  :  lieserl
46  :  lived
47  :  lwenthal
48  :  mari
49  :  married
50  :  mileva
51  :  they
52  :  three
53  :  to
54  :  together
55  :  until
56  :  18851888luitpold
57  :  18881894
58  :  18951896
59  :  18961900
60  :  1905
61  :  aarau
62  :  at
63  :  cantonal
64  :  catholic
65  :  education
66  :  elementary
67  :  federal
68  :  following
69  :  from
70  :  gymnasium
71  :  institute
72  :  main
73  :  munich
74  :  of
75  :  phd


In [12]:
y_pred = []
def retrieveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf,sentences):
    if len(question_embedding) == 0:
      question_embedding = [[(0,0)]]
    print(question_embedding)
    max_sim=-1;
    index_sim=-1;
    for index,faq_embedding in enumerate(sentence_embeddings):
        sim=cosine_similarity(faq_embedding,question_embedding)[0][0]
        print(index, sim, sentences[index])
        if sim>max_sim:
            max_sim=sim
            index_sim=index
    y_pred.append(index_sim)
       
    print("Question: ",question)
    print("Retrieved: ",FAQdf.iloc[index_sim,0]) 
    print(FAQdf.iloc[index_sim,1])        

for question in test_df['Question']:    
  retrieveAndPrintFAQAnswer(dictionary.doc2bow(clean_sentence(question).split()),bow_corpus,train_df,cleaned_sentences)

[(23, 1), (40, 1), (74, 1), (82, 1)]
0 0.04343722427630694 albert einstein was born on 14 march 1879
1 0.9615239476408232 he was born in ulm germany
2 0.9977425047798928 he died 18 april 1955 in princeton new jersey usa
3 0.9615239476408232 his father was hermann einstein and his mother was pauline einstein born koch
4 0.9977425047798928 he had one sister named maja
5 0.9130087106158283 he was married to mileva mari between 1903 and 1919 they had three children lieserl born 1902 hans albert born 1904 and eduard born 1910 he married elsa lwenthal in 1919 and they lived together until her death in 1936
6 0.9797619135918656 he received his main education at the following schools catholic elementary school in munich germany 18851888luitpold gymnasium in munich germany 18881894 cantonal school in aarau switzerland 18951896 swiss federal institute of technology in zurich switzerland 18961900 phd from zurich university switzerland 1905
7 0.9130087106158283 the nobel prize awarding institution

In [13]:
print(classification_report(y_ture, y_pred))

              precision    recall  f1-score   support

           2       0.25      1.00      0.40         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1

    accuracy                           0.20         5
   macro avg       0.05      0.20      0.08         5
weighted avg       0.05      0.20      0.08         5



## Similarity With Sentence Transformers 

In [14]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.5 MB/s 
     |████████████████████████████████| 5.5 MB 67.6 MB/s 
     |████████████████████████████████| 1.3 MB 61.3 MB/s 
     |████████████████████████████████| 182 kB 71.8 MB/s 
     |████████████████████████████████| 7.6 MB 25.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c06287af7cffebad0032ff601718af23d25e765ced85116277038cbc484e710f
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [22]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

train_sentences = [ sentence for sentence in train_df['Question']]
test_sentences = [ sentence for sentence in test_df['Question']]
train_sentence_embeddings = model.encode(train_sentences)
test_sentence_embeddings = model.encode(test_sentences)

In [23]:
for sentence, embedding in zip(train_sentences, train_sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: When was Albert Einstein born?
Embedding: [-2.51365155e-01 -3.41660023e-01  3.77101839e-01 -7.87286580e-01
 -3.70724529e-01 -4.92149442e-01  7.70790339e-01  4.51326102e-01
  4.70460951e-01 -3.84515822e-01  8.96301866e-01 -2.79865235e-01
  1.12019196e-01  3.82840753e-01  1.87249050e-01 -1.03268838e+00
 -6.02451205e-01  4.66581509e-02  4.12298054e-01 -3.99102777e-01
 -3.31107557e-01  4.76673126e-01 -2.72204995e-01 -8.51125866e-02
 -1.58944651e-01  2.23138019e-01  4.78268266e-01 -4.52732176e-01
 -1.03318132e-01 -2.44950384e-01 -4.38637473e-02  7.66097188e-01
 -2.00239465e-01  4.11086440e-01 -4.40466106e-01 -3.61289710e-01
  9.63741988e-02 -4.43248868e-01  5.38699031e-01 -4.77829814e-01
  2.86145341e-02 -3.25307280e-01  1.24696761e-01 -7.13878036e-01
 -1.49994159e+00 -4.59543496e-01 -8.83513570e-01 -3.94262075e-01
 -8.84987295e-01 -1.05996037e+00 -6.35484457e-01 -6.70913339e-01
  1.27382362e+00 -1.89638689e-01 -5.64040899e-01 -4.54550572e-02
  2.67585754e-01 -6.81751490e-01  9.04

In [24]:
for sentence, embedding in zip(test_sentences, test_sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: What is the date of his death?
Embedding: [ 7.17215240e-02 -4.51120943e-01  8.82542431e-01 -6.76571310e-01
  4.49815184e-01 -3.74043643e-01  6.54443145e-01  3.79580557e-01
  1.08548269e-01  2.03254372e-01 -9.73746926e-02 -4.39844221e-01
  2.07300991e-01 -3.20316553e-02  5.12017429e-01  1.64260432e-01
 -7.65676975e-01  5.19877598e-02 -6.17520690e-01 -6.46444499e-01
 -1.35991842e-01 -2.84840018e-01 -4.51044708e-01  9.28594768e-01
 -5.13898373e-01  9.59655106e-01  2.83969492e-01 -5.62636256e-01
 -2.70968318e-01 -4.21086848e-01  4.69562709e-01  5.70470572e-01
 -2.09958121e-01  1.79355159e-01  4.20391858e-01 -6.32008374e-01
 -4.00652409e-01 -2.47021288e-01  1.13544822e-01 -4.53936517e-01
 -3.11721973e-02 -3.42825353e-01  3.85644168e-01 -4.81224924e-01
 -3.12006652e-01 -4.58889306e-02 -4.16326523e-01  1.92265806e-03
 -9.12876785e-01 -6.35630488e-01 -4.62274961e-02  6.48815110e-02
  1.33297420e+00 -5.06840460e-02 -1.38762486e+00 -3.61470431e-01
  7.75969923e-02  2.28965078e-02 -7.73

In [ ]:
import torch.nn
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
train = torch.from_numpy(sentence_embeddings)
b = torch.from_numpy(sentence_embeddings)

In [34]:
import torch.nn
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
y_pred  = []
for target_sentence in test_df['Question']:
  cos_sum = -2.0
  idx = -1
  tar_enc = torch.from_numpy(model.encode(target_sentence))
  for i in range(len(train_df['Question'])):
    prob_enc  =  torch.from_numpy(model.encode(train_df['Question'][i]))
    curr_cos_sum  = cos(tar_enc, prob_enc).item()
    print(f'{cos_sum} : {curr_cos_sum}')
    if curr_cos_sum > cos_sum:
      cos_sum = curr_cos_sum
      idx = i
  y_pred.append(idx)

-2.0 : 0.4364027678966522
0.4364027678966522 : 0.6556687355041504
0.6556687355041504 : 0.9063027501106262
0.9063027501106262 : 0.7045108079910278
0.9063027501106262 : 0.4331781268119812
0.9063027501106262 : 0.5682768821716309
0.9063027501106262 : 0.5860550403594971
0.9063027501106262 : 0.25561338663101196
0.9063027501106262 : 0.2362281084060669
0.9063027501106262 : 0.4733584225177765
-2.0 : 0.7240618467330933
0.7240618467330933 : 0.5121322870254517
0.7240618467330933 : 0.3701789975166321
0.7240618467330933 : 0.5528684258460999
0.7240618467330933 : 0.6683369278907776
0.7240618467330933 : 0.5748536586761475
0.7240618467330933 : 0.45329850912094116
0.7240618467330933 : 0.6331170201301575
0.7240618467330933 : 0.6362589001655579
0.7240618467330933 : 0.549629807472229
-2.0 : 0.3446849584579468
0.3446849584579468 : 0.576423704624176
0.576423704624176 : 0.5964604020118713
0.5964604020118713 : 0.823780357837677
0.823780357837677 : 0.6980879306793213
0.823780357837677 : 0.6685588359832764
0.8237

In [35]:
y_pred

[2, 0, 3, 0, 6]

In [36]:
print(classification_report(y_ture, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           2       1.00      1.00      1.00         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       1.00      1.00      1.00         1

    accuracy                           0.40         5
   macro avg       0.33      0.33      0.33         5
weighted avg       0.40      0.40      0.40         5

